In [1]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten,Conv2D,Dropout
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
NUM_CLASSES = 18

IMAGE_SIZE = 120

DATASET_PATH = "data/images/pokemon_per_type"

In [3]:
# Creating the model
model=Sequential()
model.add(Convolution2D(32,(7,7),input_shape=(IMAGE_SIZE,IMAGE_SIZE,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Convolution2D(64,(4,4),input_shape=(38,38,32),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(3000,activation='relu'))
model.add(Dense(units=NUM_CLASSES,activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 114, 114, 32)      4736      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 38, 38, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 35, 35, 64)        32832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 17, 17, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 18496)             0         
                                                                 
 dense (Dense)               (None, 3000)              5

In [4]:
model.compile(optimizer= 'adam', loss= 'categorical_crossentropy', metrics= ['accuracy'])


In [5]:
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   shear_range = 0.2, 
                                   zoom_range = 0.2, 
                                   rotation_range = 20,)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(DATASET_PATH + '/train', 
                                                    target_size = (IMAGE_SIZE, IMAGE_SIZE), 
                                                    batch_size = 32)
test_set = test_datagen.flow_from_directory(DATASET_PATH + '/val',
                                                target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                 batch_size = 32)

Found 944 images belonging to 18 classes.
Found 270 images belonging to 18 classes.


In [ ]:
fitted_model = model.fit(training_set,
                        steps_per_epoch = 10,
                        epochs = 50,
                        validation_data = test_set,
                        shuffle = 0,
                        validation_steps = 2)

In [ ]:
plt.figure()
plt.plot([i for i in range(100)],fitted_model.history['accuracy'], label = "Train Accuracy")
plt.plot([i for i in range(100)],fitted_model.history['val_accuracy'], label = "Val Accuracy")
plt.legend()

In [ ]:
plt.figure()
plt.plot([i for i in range(100)],fitted_model.history['loss'], label = "Loss")
plt.plot([i for i in range(100)],fitted_model.history['val_loss'], label = "Loss")
plt.legend()

In [ ]:
def pred(image_path):
    image = Image.open(image_path)
    image = image.resize((128,128))
    x = np.array(image)
    x = np.expand_dims(x,axis=0)
    c = np.argmax(model.predict(x), axis=1)
    l = os.listdir(DATASET_PATH + '/train')
    l.sort()
    return l[c[0]]


pred(DATASET_PATH + '/Dragon/dialga.png')